In [1]:
import xml.etree.ElementTree as ET

def read_anntation(xml_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    bounding_box_list = []

    file_name = root.find('filename').text
    for obj in root.iter('object'):

        object_label = obj.find("name").text
        for box in obj.findall("bndbox"):
            x_min = int(box.find("xmin").text)
            y_min = int(box.find("ymin").text)
            x_max = int(box.find("xmax").text)
            y_max = int(box.find("ymax").text)

        bounding_box = [object_label, x_min, y_min, x_max, y_max]
        bounding_box_list.append(bounding_box)

    return bounding_box_list, file_name

In [2]:
from os import listdir
import cv2
import numpy as np

def read_train_dataset(dir):
    images = []
    annotations = []

    for file in listdir(dir):
        if 'jpg' in file.lower() or 'png' in file.lower():
            images.append(cv2.imread(dir + file, 1))
            annotation_file = file.replace(file.split('.')[-1], 'xml')
            bounding_box_list, file_name = read_anntation(dir + annotation_file)
            annotations.append((bounding_box_list, annotation_file, file_name))

    images = np.array(images)

    return images, annotations

In [4]:
!pip install files

  Created wheel for files: filename=files-1.1.1-py3-none-any.whl size=3670 sha256=f591efe57a106bdfa13ced93ed79e2a26b2a8919d22429dc2e9e6532661b148e
  Stored in directory: c:\users\luvu1\appdata\local\pip\cache\wheels\49\93\8d\577f016b6575dbe61c8f2de6fec2a4a9c94eccb2bb3655cd50
Successfully built files


In [9]:
import imgaug as ia
from imgaug import augmenters as iaa
from files import *
from pascal_voc_writer import Writer

ia.seed(1)

dir = 'C:/Users/luvu1/Documents/data_analysis/DeepLearning_Project/100_People_Multi-view_Tracking_Data (1)/data/Multi-view_tracking_indoor/00001_market/frames/xmlfile/'
images, annotations = read_train_dataset(dir)

for idx in range(len(images)):
    image = images[idx]
    boxes = annotations[idx][0]

    ia_bounding_boxes = []
    for box in boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

    seq = iaa.Sequential([
        iaa.Multiply((1.2, 1.5)),
        iaa.Affine(
            translate_px={"x": 40, "y": 60},
            scale=(0.5, 0.7),
            rotate=45
        )
    ])

    seq_det = seq.to_deterministic()

    image_aug = seq_det.augment_images([image])[0]
    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

    new_image_file = dir + 'after_' + annotations[idx][2]
    cv2.imwrite(new_image_file, image_aug)

    h, w = np.shape(image_aug)[0:2]
    voc_writer = Writer(new_image_file, w, h)

    for i in range(len(bbs_aug.bounding_boxes)):
        bb_box = bbs_aug.bounding_boxes[i]
        voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

    voc_writer.save(dir + 'after_' + annotations[idx][1])

In [8]:
!pip install pascal-voc-writer

In [11]:
import imgaug as ia
ia.seed(1)
# imgaug uses matplotlib backend for displaying images
%matplotlib inline
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from imgaug import augmenters as iaa 
# imageio library will be used for image input/output
import imageio
import pandas as pd
import numpy as np
import re
import os
import glob
# this library is needed to read XML files for converting it into CSV
import xml.etree.ElementTree as ET
import shutil

In [29]:
# Function that will extract column data for our CSV file as pandas DataFrame
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            try:
                value = (root.find('filename').text,
                         int(root.find('size')[0].text),
                         int(root.find('size')[1].text),
                         member[0].text,
                         int(member[4][0].text),
                         int(member[4][1].text),
                         int(member[4][2].text),
                         int(member[4][3].text)
                         )
                xml_list.append(value)
            except:
                pass
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df
   
# apply the function to convert all XML files in images/ folder into labels.csv
labels_df = xml_to_csv('C:/Users/luvu1/Documents/data_analysis/DeepLearning_Project/xmlfile/')
# labels_df.to_csv('labels.txt', index=False, header=None, sep="\t")

In [16]:
labels_df.to_csv('C:/Users/luvu1/Documents/data_analysis/DeepLearning_Project/xmlfile/labels.txt', index=False, header=None, sep="\t")

In [30]:
labels_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,01_01.jpg,1920,1080,person,1074,1,1174,197
1,01_02.jpg,1920,1080,person,1074,2,1166,200
2,01_03.jpg,1920,1080,person,977,24,1041,200
3,01_04.jpg,1920,1080,person,839,80,937,306
4,01_05.jpg,1920,1080,person,686,138,833,414


In [33]:
num = (labels_df['xmin'] + labels_df['xmax']) / 2
labels_df['center_x'] = num / labels_df['width']

In [38]:
num = (labels_df['ymin'] + labels_df['ymax']) / 2
labels_df['center_y'] = num / labels_df['height']

In [39]:
num = (labels_df['xmax'] - labels_df['xmin'])
labels_df['w'] = num / labels_df['width']

In [40]:
num = (labels_df['ymax'] - labels_df['ymin'])
labels_df['h'] = num / labels_df['height']
labels_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,center_x,center_y,w,h
0,01_01.jpg,1920,1080,person,1074,1,1174,197,0.585417,0.091667,0.052083,0.181481
1,01_02.jpg,1920,1080,person,1074,2,1166,200,0.583333,0.093519,0.047917,0.183333
2,01_03.jpg,1920,1080,person,977,24,1041,200,0.525521,0.103704,0.033333,0.162963
3,01_04.jpg,1920,1080,person,839,80,937,306,0.462500,0.178704,0.051042,0.209259
4,01_05.jpg,1920,1080,person,686,138,833,414,0.395573,0.255556,0.076563,0.255556


In [52]:
for idx in range(len(labels_df)):
  data = labels_df.iloc[idx]
  filename = data['filename'].split('.')[0]
  f = open('C:/Users/luvu1/Documents/data_analysis/DeepLearning_Project/100_People_Multi-view_Tracking_Data (1)/data/Multi-view_tracking_indoor/00001_market/frames/training/{}.txt'.format(filename), 'w')
  wdata = str("0 {} {} {} {}".format(data['center_x'], data['center_y'], data['w'], data['h']))
  f.write(wdata)
  f.close()

In [46]:
  data = labels_df.iloc[0]
  f = open("new.txt", 'w')
  wdata = str("0 {} {} {} {}".format(data['center_x'], data['center_y'], data['w'], data['h']))
  f.write(wdata)
  f.close()

In [ ]:
C:\Users\luvu1\Documents\data_analysis\DeepLearning_Project\100_People_Multi-view_Tracking_Data (1)\data\Multi-view_tracking_indoor\00001_market\frames\training